In [1]:
'''
QKD - BB84
    1. select encoding: Alice randomly selects a basis ( × or + ) to encode each bit.
    2. select measurement: Bob randomly selects a basis ( × or + ) to measure each bit.
    3. encode: Alice creates the quantum states, encoded in the selected bases.
    4. send: Alice sends Bob the encoded states, via the quantum channel.
    5. measure: Bob measures all the quantum states in his pre-selected measurement bases.
    6. announce: Alice announces which basis she used to encode each bit, via the classical channel.
    7. create key: Alice and Bob discard bits in their key that used a different encoding and decoding basis.
'''

'\nQKD - BB84\n    1. select encoding: Alice randomly selects a basis ( × or + ) to encode each bit.\n    2. select measurement: Bob randomly selects a basis ( × or + ) to measure each bit.\n    3. encode: Alice creates the quantum states, encoded in the selected bases.\n    4. send: Alice sends Bob the encoded states, via the quantum channel.\n    5. measure: Bob measures all the quantum states in his pre-selected measurement bases.\n    6. announce: Alice announces which basis she used to encode each bit, via the classical channel.\n    7. create key: Alice and Bob discard bits in their key that used a different encoding and decoding basis.\n'

In [2]:
from random import getrandbits
import binascii
from qiskit import QuantumCircuit, Aer, execute

In [3]:
def construct_key_from_indices(bitstring, indices):
    key = ''
    for idx in indices:
        # if bases match, the bitstring bit is added to the key
        key = key + bitstring[idx] 
    return key

In [4]:
def bob_compare_bases(alices_bases, bobs_bases):
    indices = []
    
    for i in range(len(alices_bases)):
        if alices_bases[i] == bobs_bases[i]:
            indices.append(i)
    return indices

In [5]:
def measure(bob_bases, encoded_qubits, backend):
    bob_bitstring = ''
    
    for i in range(len(encoded_qubits)):
        qc = encoded_qubits[i]
        
        if bob_bases[i] == "0":
            # 0 = Z basis
            qc.measure(0, 0)

        elif bob_bases[i] == "1":
            # 1 = X basis
            qc.h(0)
            qc.measure(0, 0)
        
        # Run the circuit
        job = execute(qc, backend=backend, shots = 1)
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bob_bitstring += measured_bit 
        
    return bob_bitstring

In [6]:
def encode(alice_bitstring, alice_bases):
    encoded_qubits = []
    for i in range(len(alice_bitstring)):
        qc = QuantumCircuit(1, 1)

        if alice_bases[i] == "0":
            # 0 Means we are encoding in the z basis
            if alice_bitstring[i] == "0":
                pass # |0>
            
            elif alice_bitstring[i] == "1":
                qc.x(0) # |1>
                
        elif alice_bases[i] == "1":
            # 1 Means we are encoding in the x basis
            if alice_bitstring[i] == "0":
                qc.h(0) # |+>
            elif alice_bitstring[i] == "1":
                qc.x(0)
                qc.h(0) # |->
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
    return encoded_qubits

In [7]:
def select_measurement(length):
    bob_bases = ""
    
    for i in range(length):
        bob_bases += (str(getrandbits(1)))

    return bob_bases

In [8]:
# 0 = (0, 1), 1 = (+, -) basis
def select_encoding(length):

    alice_bitstring = ""
    alice_bases = ""

    for i in range(length):
        alice_bitstring += (str(getrandbits(1)))
        alice_bases += (str(getrandbits(1)))

    return alice_bitstring, alice_bases

In [9]:
KEY_LENGTH = 500

alice_bitstring, alice_bases = select_encoding(KEY_LENGTH)

bob_bases = select_measurement(KEY_LENGTH)

encoded_qubits = encode(alice_bitstring, alice_bases)

QUANTUM_CHANNEL = encoded_qubits

bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, Aer.get_backend('qasm_simulator'))

CLASSICAL_CHANNEL = alice_bases
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)


CLASSICAL_CHANNEL = agreeing_bases

alice_key = construct_key_from_indices(alice_bitstring, CLASSICAL_CHANNEL)
bob_key = construct_key_from_indices(bob_bitstring, agreeing_bases)

# Preview the first 10 elements of each Key:
print("alice_key: ", alice_key[:10])
print("bob_key: ", bob_key[:10])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)

alice_key:  0010100000
bob_key:  0010100000
Alice's key is equal to Bob's key:  True


In [10]:
import binascii

def encrypt_message(unencrypted_string, key):
    # ASCII to Binary
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))

    # Encrypt binary
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )

    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # Unencrypt binary
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Binary to ASCII
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

In [11]:
message = "Welcome To Hell!"

encrypted_message = encrypt_message(message, alice_key)
print("Encrypted message:", encrypted_message)

decrypted_message = decrypt_message(encrypted_message, bob_key)
print("\nDecrypted message:", decrypted_message)

if message == decrypted_message:
    print("\nMessage Transfer Successful!")

Encrypted message: 01111111010010110001110100010010010011010010001010001000100010110010010001001100011101011100111010111011010111000110001000010010

Decrypted message: Welcome To Hell!

Message Transfer Successful!
